## Considere a base de dados **Customer Data and Bank Loans**
- https://www.kaggle.com/datasets/daniellopez01/credit-risk
- Formulário para envio da Atividade: https://forms.gle/5jCooRKSFvKDvtGG6

### Exercícios:
---

1. **Agrupamento por `credit_history` com resumo estatístico:**  
   Agrupe os dados pela coluna `credit_history` e calcule:
   - A soma total dos valores da coluna `amount`;
   - A idade média (`age`) para cada grupo.

---

2. **Criação de uma coluna com média ajustada:**  
   Crie uma nova coluna chamada `amount_ratio`, que contém o valor de `amount` dividido pela média de `amount` em cada categoria de `purpose`.

---

3. **Classificação de idade:**  
   Crie uma nova coluna chamada `age_group` que classifica os valores de `age` em categorias:
   - "Under 30" se a idade for menor que 30;
   - "30-50" se a idade estiver entre 30 e 50;
   - "Over 50" se a idade for maior que 50.

---

4. **Análise de variabilidade por `employment_duration` e `housing`:**  
   Agrupe os dados pelas colunas `employment_duration` e `housing` e calcule:
   - A média e o desvio padrão (`std`) da coluna `amount`.

---

5. **Distribuição de idade mínima e máxima por `job`:**  
   Agrupe os dados pela coluna `job` e calcule:
   - A idade mínima (`age_min`) e a idade máxima (`age_max`) para cada categoria de emprego.

---

6. **Diferença entre o valor do empréstimo e a média por histórico de crédito (`transform`):**  
   Crie uma nova coluna chamada `amount_diff`, que contém a diferença entre o valor de `amount` e a média de `amount` dentro de cada grupo de `credit_history`.

---

7. **Quantidade de dependentes por tipo de conta bancária (`groupby` e `agg`):**  
   Agrupe os dados pela coluna `checking_balance` e calcule:
   - A soma total de `dependents`;
   - O número médio de dependentes por categoria de conta bancária.

---


In [6]:
import kagglehub
path = kagglehub.dataset_download("daniellopez01/credit-risk")

In [7]:
print(f'{path}\\credit.csv')

C:\Users\e101152\.cache\kagglehub\datasets\daniellopez01\credit-risk\versions\1\credit.csv


In [3]:
import pandas as pd

In [18]:
arquivo = pd.read_csv(f'{path}\\credit.csv')
arquivo.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes


In [19]:
### Resposta 1: Agrupando por credit_history
grouped = arquivo.groupby('credit_history')

In [20]:
### Cálculo das soma de amount e média da idade
grouped.agg(
    total_amount=('amount', 'sum'),
    avg_age=('age', 'mean') 
).reset_index()

,credit_history,total_amount,avg_age
0,critical,904795,38.436860
1,good,1611708,33.877358
2,perfect,212227,34.300000
3,poor,378629,36.136364
4,very good,163899,36.265306


In [22]:
### Resposta 2: Criação da coluna amount_ratio, sendo ela amount dividida pela média de amount por purpose
mean_amount_by_purpose = arquivo.groupby('purpose')['amount'].transform('mean')
arquivo['amount_ratio'] = arquivo['amount'] / mean_amount_by_purpose
arquivo.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default,amount_ratio
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no,0.435506
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes,2.217018
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no,0.727979
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no,2.936403
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes,1.292397


In [30]:
### Resposta 3: Classificando os valores das idades nos seguintes grupos "'Under 30', '30-50', 'Over 50'"
result = arquivo['age_group'] = pd.cut(
    arquivo['age'],
    bins=[-float('inf'), 30, 50, float('inf')],
    labels=['Under 30', '30-50', 'Over 50']
)
result.head()

0     Over 50
1    Under 30
2       30-50
3       30-50
4     Over 50
Name: age, dtype: category
Categories (3, object): ['Under 30' < '30-50' < 'Over 50']

In [26]:
# Resposta 4: Agrupando pelas colunas employment_duration e housing e calculando média e destio padrão de amount. 
result = arquivo.groupby(['employment_duration', 'housing']).agg(
    mean_amount=('amount', 'mean'),
    std_amount=('amount', 'std') 
).reset_index()
result.head()

,employment_duration,housing,mean_amount,std_amount
0,1 - 4 years,other,5112.961538,3554.197468
1,1 - 4 years,own,2999.162698,2590.607900
2,1 - 4 years,rent,2799.114754,2339.122571
3,4 - 7 years,other,4788.454545,3254.082770
4,4 - 7 years,own,3556.053435,3101.828623


In [27]:
# Resposta 5: Agrupando por job e calculando a idade máxima e minida de cada job.
result = arquivo.groupby('job').agg(
    age_min=('age', 'min'), 
    age_max=('age', 'max') 
).reset_index()

result.head()

,job,age_min,age_max
0,management,20,75
1,skilled,19,68
2,unemployed,23,74
3,unskilled,19,74


In [31]:
# Resposta 6: Cálculei a média de amout agrupado por credit_history e depois criei uma nova coluna substraindo a média de amount
mean_amount_by_credit_history = arquivo.groupby('credit_history')['amount'].transform('mean')
arquivo['amount_diff'] = arquivo['amount'] - mean_amount_by_credit_history
arquivo.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default,amount_ratio,age_group,amount_diff
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no,0.435506,Over 50,-1919.037543
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes,2.217018,Under 30,2910.041509
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no,0.727979,30-50,-992.037543
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no,2.936403,30-50,4841.041509
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes,1.292397,Over 50,567.397727


In [33]:
# Resposta 7: Agrupei por checking_balance e somei o total de dependents e também calculei a médica de dependents por categoria utilizando agg. 
result = arquivo.groupby('checking_balance').agg(
    sum_dependents=('dependents', 'sum'),
    mean_dependents=('dependents', 'mean') 
)
result.head()

,sum_dependents,mean_dependents
checking_balance,,
1 - 200 DM,300,1.115242
< 0 DM,326,1.189781
> 200 DM,72,1.142857
unknown,457,1.159898
